In [ ]:
#To export to Word/google doc, first export notebook as python, then use this tool to generate copieable syntax highlighted code: http://www.planetb.ca/syntax-highlight-word
import numpy as np
from enum import Enum

In [ ]:
#constants
num_rows = 3
num_cols = 4
prob_going_in_correct_dir = 0.8
prob_moving_at_right_angles_of_intended_direction = 0.1
terminal_states = [(4,2),(4,3)]
empty_states = [(2,2)]
class Walls(Enum):
   ONE = 1
   TWO = 2
   END = 3

class Actions(Enum):
   UP = 1
   DOWN = 2
   LEFT = 3
   RIGHT = 4

In [ ]:
#returns the element in grid that is a np array when given col and row coordinates given in lecture
def getGridElement(grid,col,row):
  return grid[(len(grid) - row,col-1)]

def updateGridElement(grid,col,row,value):
  grid[(len(grid) - row,col-1)] = value

In [ ]:
def isValid(state):
  col,row = state
  if col<=0 or col >num_cols or row<=0 or row>num_rows:
    return False
  if state in empty_states:
    return False
  return True

def get_left_right_up_down_neighbors(state):
  col,row = state
  return [(col-1,row),(col+1,row),(col,row+1),(col,row-1)]

def get_num_walls_around_state(state):
  num_walls = 0
  for neighbor in get_left_right_up_down_neighbors(state):
    if not isValid(neighbor):
      num_walls+=1
  return num_walls

In [ ]:
def get_prob_s_prime_given_a_s(s_prime, a, s):
  if (s in get_left_right_up_down_neighbors(s_prime) or s == s_prime) and isValid(s) and (s not in terminal_states):
    left_neighbor,right_neighbor,up_neighbor,down_neighbor = get_left_right_up_down_neighbors(s_prime)

    if a==Actions.UP:
      if s==left_neighbor or  s==right_neighbor:
        return prob_moving_at_right_angles_of_intended_direction
      elif s==down_neighbor:
        return prob_going_in_correct_dir
      elif s == s_prime:
        prob = 0
        if not isValid((s[0],s[1]+1)):
          prob+=prob_going_in_correct_dir
        if not isValid((s[0]-1,s[1])):
          prob+=prob_moving_at_right_angles_of_intended_direction
        if not isValid((s[0]+1,s[1])):
          prob+=prob_moving_at_right_angles_of_intended_direction
        return prob
      else:
        return 0
    
    if a==Actions.DOWN:
      if s==left_neighbor or  s==right_neighbor:
        return prob_moving_at_right_angles_of_intended_direction
      elif s==up_neighbor:
        return prob_going_in_correct_dir
      elif s == s_prime:
        prob = 0
        if not isValid((s[0],s[1]-1)):
          prob+=prob_going_in_correct_dir
        if not isValid((s[0]-1,s[1])):
          prob+=prob_moving_at_right_angles_of_intended_direction
        if not isValid((s[0]+1,s[1])):
          prob+=prob_moving_at_right_angles_of_intended_direction
        return prob
      else:
        return 0

    if a==Actions.LEFT:
      if s==up_neighbor or  s==down_neighbor:
        return prob_moving_at_right_angles_of_intended_direction
      elif s==right_neighbor:
        return prob_going_in_correct_dir
      elif s == s_prime:
        prob = 0
        if not isValid((s[0]-1,s[1])):
          prob+=prob_going_in_correct_dir
        if not isValid((s[0],s[1]+1)):
          prob+=prob_moving_at_right_angles_of_intended_direction
        if not isValid((s[0]+1,s[1]-1)):
          prob+=prob_moving_at_right_angles_of_intended_direction
        return prob
      else:
        return 0

    if a==Actions.RIGHT:
      if s==up_neighbor or  s==down_neighbor:
        return prob_moving_at_right_angles_of_intended_direction
      elif s==left_neighbor:
        return prob_going_in_correct_dir
      elif s == s_prime:
        prob = 0
        if not isValid((s[0]+1,s[1])):
          prob+=prob_going_in_correct_dir
        if not isValid((s[0],s[1]+1)):
          prob+=prob_moving_at_right_angles_of_intended_direction
        if not isValid((s[0]+1,s[1]-1)):
          prob+=prob_moving_at_right_angles_of_intended_direction
        return prob
      else:
        return 0
  else:
    return 0



In [ ]:
def get_prob_e_given_s_prime(e,s_prime):
  col,row = s_prime 
  #terminal 
  if s_prime == (4,3) or s_prime == (4,2):
    if e == Walls.END:
      return 1
    else:
      return 0
  
  #non terminal in third column
  if col ==3:
    if e==Walls.ONE:
      return 0.9
    elif e==Walls.TWO:
      return 0.1
    elif e==Walls.END:
      return 0
  
  # all other columns
  if e==Walls.ONE:
    return 0.1
  elif e==Walls.TWO:
    return 0.9
  elif e==Walls.END:
    return 0

In [ ]:
def updateSingleBeliefState(b_s,a,e):
  new_b_s = b_s.copy()
  
  for row in range(1, len(new_b_s)+1):
    for col in range(1, len(new_b_s[0])+1):
      if isValid((col,row)):
        sum_term = 0
        for neighbor in get_left_right_up_down_neighbors((col,row)) + [(col,row)]:
          if isValid(neighbor):
            sum_term += get_prob_s_prime_given_a_s((col,row), a, neighbor) * getGridElement(b_s,neighbor[0],neighbor[1])
        
        updateGridElement(new_b_s,col,row, get_prob_e_given_s_prime(e,(col,row))*sum_term)
  
  #return new_b_s #temp for testing to delete
  return new_b_s/(sum(sum(new_b_s))).copy()



In [ ]:
def updateBeliefState(b_s,a1_n,e1_n):
  for a,e in zip(a1_n,e1_n):
    print(b_s)
    print("\n")
    b_s = updateSingleBeliefState(b_s,a,e)
  return (b_s)

### Tests

In [ ]:
def get_uniform_belief_state_on_non_terminal_states():
  b_s = np.zeros((3,4))
  for row in range(1, len(b_s)+1):
    for col in range(1, len(b_s[0])+1):
      if isValid((col,row)) and (col,row) not in terminal_states:
        updateGridElement(b_s,col,row, 1)

  b_s = b_s/(sum(sum(b_s)))
  return b_s


In [ ]:
# (up, up , up) (2,2,2)
b_s = get_uniform_belief_state_on_non_terminal_states()
a1_n = (Actions.UP, Actions.UP, Actions.UP)
e1_n = (Walls.TWO,Walls.TWO,Walls.TWO)
res = updateBeliefState(b_s,a1_n,e1_n)
print(res)

In [ ]:
#(up, up, up) (1,1,1)
b_s = get_uniform_belief_state_on_non_terminal_states()
a1_n = (Actions.UP, Actions.UP, Actions.UP)
e1_n = (Walls.ONE,Walls.ONE,Walls.ONE)
res = updateBeliefState(b_s,a1_n,e1_n)
print(res)

In [ ]:
#(right, right, up) (1,1,end) with S0 = (2,3)
b_s = np.zeros((3,4))
b_s[(0,1)] = 1
a1_n = (Actions.RIGHT, Actions.RIGHT, Actions.UP)
e1_n = (Walls.ONE,Walls.ONE,Walls.END)
res = updateBeliefState(b_s,a1_n,e1_n)
print(res)
print("Output makes sense as we look at the breakdown of each step.")

In [ ]:
#(up, right, right, right) (2,2,1,1) with S0 = (1,1)
b_s = np.zeros((3,4))
b_s[(2,0)] = 1
a1_n = (Actions.UP, Actions.RIGHT, Actions.RIGHT,Actions.RIGHT)
e1_n = (Walls.TWO,Walls.TWO,Walls.ONE,Walls.ONE)
res = updateBeliefState(b_s,a1_n,e1_n)
print(res)

In [ ]:
import unittest
print("d")
class TestNotebook(unittest.TestCase):

  def test_getGridElement(self):
    grid = np.array([[1,2,3],[4,5,6]])
    self.assertEqual(getGridElement(grid,1,1),4)
    self.assertEqual(getGridElement(grid,1,2),1)
    self.assertEqual(getGridElement(grid,2,1),5)
    self.assertEqual(getGridElement(grid,3,2),3)

  def test_get_prob_e_given_s_prime(self):
    self.assertEqual(get_prob_e_given_s_prime(Walls.ONE,(1,1)), 0.1)
    self.assertEqual(get_prob_e_given_s_prime(Walls.TWO,(1,1)), 0.9)
    self.assertEqual(get_prob_e_given_s_prime(Walls.ONE,(3,1)), 0.9)
    self.assertEqual(get_prob_e_given_s_prime(Walls.END,(4,3)), 1)
    self.assertEqual(get_prob_e_given_s_prime(Walls.END,(4,2)), 1)

  def test_updateBeliefState_beforeAnyObservation_uniformProbabilityReturned(self):
    initial_belief_state = np.ones((num_rows,num_cols))*(1/(num_rows*num_cols))

    self.assertEqual(updateBeliefState(initial_belief_state,[],[]), initial_belief_state)
  unittest.main(argv=[''], verbosity=2, exit=False)